segunda modificacion de gestor de tareas , se le añadio la vnetana de bienvenida 

In [1]:
"""
Task Manager - Gestor de Tareas Personales
Python 3.x + Tkinter (ttk) + SQLite

Descripción:
    Aplicación de escritorio para gestionar tareas personales.
    - Guardado persistente en SQLite (tasks.db)
    - Interfaz con Treeview, formularios, búsqueda, filtros
"""

import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from datetime import datetime

DB_FILE = "tasks.db"

# ---------- Funciones auxiliares ----------
def validar_fecha(fecha_texto):
    """Valida formato YYYY-MM-DD. Devuelve True/False."""
    try:
        datetime.strptime(fecha_texto, "%Y-%m-%d")
        return True
    except Exception:
        return False

# ---------- Capa de Datos (SQLite) ----------
class TaskDB:
    def __init__(self, db_path=DB_FILE):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.create_table()

    def create_table(self):
        cursor = self.conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            description TEXT,
            due_date TEXT,
            priority TEXT,
            status TEXT,
            created_at TEXT
        )
        """)
        self.conn.commit()

    def add_task(self, title, description, due_date, priority, status):
        cursor = self.conn.cursor()
        cursor.execute("""
            INSERT INTO tasks (title, description, due_date, priority, status, created_at)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (title, description, due_date, priority, status, datetime.now().isoformat()))
        self.conn.commit()
        return cursor.lastrowid

    def update_task(self, task_id, title, description, due_date, priority, status):
        cursor = self.conn.cursor()
        cursor.execute("""
            UPDATE tasks
            SET title=?, description=?, due_date=?, priority=?, status=?
            WHERE id=?
        """, (title, description, due_date, priority, status, task_id))
        self.conn.commit()

    def delete_task(self, task_id):
        cursor = self.conn.cursor()
        cursor.execute("DELETE FROM tasks WHERE id=?", (task_id,))
        self.conn.commit()

    def get_all_tasks(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, title, description, due_date, priority, status FROM tasks ORDER BY id DESC")
        return cursor.fetchall()

    def get_task(self, task_id):
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, title, description, due_date, priority, status FROM tasks WHERE id=?", (task_id,))
        return cursor.fetchone()

    def search_tasks(self, query):
        cursor = self.conn.cursor()
        q = f"%{query}%"
        cursor.execute("""
            SELECT id, title, description, due_date, priority, status
            FROM tasks
            WHERE title LIKE ? OR description LIKE ?
            ORDER BY id DESC
        """, (q, q))
        return cursor.fetchall()

    def close(self):
        self.conn.close()

# ---------- Ventana Formulario (Agregar / Editar) ----------
class TaskForm(tk.Toplevel):
    def __init__(self, master, db: TaskDB, on_save_callback, task_id=None):
        super().__init__(master)
        self.title("Agregar Tarea" if task_id is None else "Editar Tarea")
        self.resizable(False, False)
        self.db = db
        self.on_save_callback = on_save_callback
        self.task_id = task_id

        pad = 8
        self.columnconfigure(1, weight=1)
        ttk.Label(self, text="Título:*").grid(row=0, column=0, padx=pad, pady=pad, sticky="e")
        self.title_var = tk.StringVar()
        self.title_entry = ttk.Entry(self, textvariable=self.title_var, width=50)
        self.title_entry.grid(row=0, column=1, padx=pad, pady=pad, sticky="we")

        ttk.Label(self, text="Descripción:").grid(row=1, column=0, padx=pad, pady=pad, sticky="ne")
        self.desc_text = tk.Text(self, height=5, width=50)
        self.desc_text.grid(row=1, column=1, padx=pad, pady=pad, sticky="we")

        ttk.Label(self, text="Fecha vencimiento (YYYY-MM-DD):").grid(row=2, column=0, padx=pad, pady=pad, sticky="e")
        self.due_var = tk.StringVar()
        self.due_entry = ttk.Entry(self, textvariable=self.due_var)
        self.due_entry.grid(row=2, column=1, padx=pad, pady=pad, sticky="we")

        ttk.Label(self, text="Prioridad:").grid(row=3, column=0, padx=pad, pady=pad, sticky="e")
        self.priority_var = tk.StringVar()
        self.priority_cb = ttk.Combobox(self, textvariable=self.priority_var, values=["Baja", "Media", "Alta"], state="readonly")
        self.priority_cb.grid(row=3, column=1, padx=pad, pady=pad, sticky="we")
        self.priority_cb.current(1)

        ttk.Label(self, text="Estado:").grid(row=4, column=0, padx=pad, pady=pad, sticky="e")
        self.status_var = tk.StringVar()
        self.status_cb = ttk.Combobox(self, textvariable=self.status_var, values=["Pendiente", "En progreso", "Completada"], state="readonly")
        self.status_cb.grid(row=4, column=1, padx=pad, pady=pad, sticky="we")
        self.status_cb.current(0)

        btn_frame = ttk.Frame(self)
        btn_frame.grid(row=5, column=0, columnspan=2, pady=(0,pad))
        save_btn = ttk.Button(btn_frame, text="Guardar", command=self.guardar)
        save_btn.pack(side="left", padx=5)
        cancel_btn = ttk.Button(btn_frame, text="Cancelar", command=self.destroy)
        cancel_btn.pack(side="left")

        if self.task_id:
            self.load_task()

    def load_task(self):
        record = self.db.get_task(self.task_id)
        if record:
            _id, title, description, due_date, priority, status = record
            self.title_var.set(title)
            self.desc_text.delete("1.0", tk.END)
            self.desc_text.insert(tk.END, description or "")
            self.due_var.set(due_date or "")
            if priority:
                self.priority_var.set(priority)
            if status:
                self.status_var.set(status)

    def validar(self):
        title = self.title_var.get().strip()
        if not title:
            messagebox.showwarning("Validación", "El título es obligatorio.")
            return False
        due = self.due_var.get().strip()
        if due and not validar_fecha(due):
            messagebox.showwarning("Validación", "La fecha debe tener formato YYYY-MM-DD.")
            return False
        return True

    def guardar(self):
        if not self.validar():
            return
        title = self.title_var.get().strip()
        description = self.desc_text.get("1.0", tk.END).strip()
        due = self.due_var.get().strip() or None
        priority = self.priority_var.get()
        status = self.status_var.get()
        if self.task_id:
            self.db.update_task(self.task_id, title, description, due, priority, status)
            messagebox.showinfo("Éxito", "Tarea actualizada correctamente.")
        else:
            self.db.add_task(title, description, due, priority, status)
            messagebox.showinfo("Éxito", "Tarea agregada correctamente.")
        self.on_save_callback()
        self.destroy()

# ---------- Ventana Acerca/Estadísticas ----------
class AboutWindow(tk.Toplevel):
    def __init__(self, master, db: TaskDB):
        super().__init__(master)
        self.title("Acerca y Estadísticas")
        self.resizable(False, False)
        self.db = db

        ttk.Label(self, text="Gestor de Tareas Personales", font=("TkDefaultFont", 12, "bold")).pack(padx=12, pady=8)
        ttk.Separator(self, orient="horizontal").pack(fill="x", pady=8, padx=8)

        stats_frame = ttk.Frame(self)
        stats_frame.pack(padx=12, pady=6)
        tasks = self.db.get_all_tasks()
        total = len(tasks)
        completadas = sum(1 for t in tasks if t[5] == "Completada")
        pendientes = total - completadas

        ttk.Label(stats_frame, text=f"Total de tareas: {total}").grid(row=0, column=0, sticky="w")
        ttk.Label(stats_frame, text=f"Tareas completadas: {completadas}").grid(row=1, column=0, sticky="w")
        ttk.Label(stats_frame, text=f"Tareas pendientes: {pendientes}").grid(row=2, column=0, sticky="w")

        ttk.Button(self, text="Cerrar", command=self.destroy).pack(pady=10)

# ---------- Ventana Principal ----------
class TaskManagerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Gestor de Tareas Personales")
        self.geometry("850x520")
        self.minsize(700, 420)
        self.db = TaskDB()
        self.style = ttk.Style(self)
        try:
            self.style.theme_use("clam")
        except Exception:
            pass
        self.create_widgets()
        self.load_tasks()

    def create_widgets(self):
        top_frame = ttk.Frame(self)
        top_frame.pack(fill="x", padx=8, pady=8)

        ttk.Label(top_frame, text="Buscar:").pack(side="left")
        self.search_var = tk.StringVar()
        search_entry = ttk.Entry(top_frame, textvariable=self.search_var, width=30)
        search_entry.pack(side="left", padx=(5,3))
        search_entry.bind("<Return>", lambda e: self.search_tasks())
        ttk.Button(top_frame, text="Ir", command=self.search_tasks).pack(side="left", padx=3)
        ttk.Button(top_frame, text="Mostrar todo", command=self.load_tasks).pack(side="left", padx=5)
        ttk.Label(top_frame, text="   ").pack(side="left", expand=True)
        ttk.Button(top_frame, text="Agregar", command=self.open_add_form).pack(side="right", padx=4)
        ttk.Button(top_frame, text="Editar", command=self.open_edit_form).pack(side="right", padx=4)
        ttk.Button(top_frame, text="Eliminar", command=self.delete_selected).pack(side="right", padx=4)
        ttk.Button(top_frame, text="Marcar Completada", command=self.mark_completed).pack(side="right", padx=4)
        ttk.Button(top_frame, text="Acerca/Stats", command=self.open_about).pack(side="right", padx=4)

        cols = ("id","title","due","priority","status")
        self.tree = ttk.Treeview(self, columns=cols, show="headings", selectmode="browse")
        for c,h in zip(cols, ["ID","Título","Vencimiento","Prioridad","Estado"]):
            self.tree.heading(c, text=h)
        self.tree.column("id", width=50, anchor="center")
        self.tree.column("title", width=360, anchor="w")
        self.tree.column("due", width=100, anchor="center")
        self.tree.column("priority", width=80, anchor="center")
        self.tree.column("status", width=120, anchor="center")
        self.tree.pack(fill="both", expand=True, padx=8, pady=(0,8))

        bottom_frame = ttk.Frame(self)
        bottom_frame.pack(fill="x", padx=8, pady=(0,8))
        self.info_label = ttk.Label(bottom_frame, text="Seleccione una tarea para ver detalles.")
        self.info_label.pack(side="left")

        self.tree.bind("<Double-1>", lambda e: self.open_edit_form())

    def load_tasks(self):
        for r in self.tree.get_children():
            self.tree.delete(r)
        rows = self.db.get_all_tasks()
        for row in rows:
            tid, title, desc, due, priority, status = row
            self.tree.insert("", tk.END, values=(tid, title, due or "", priority or "", status or ""))
        self.info_label.config(text=f"Tareas cargadas: {len(rows)}")

    def search_tasks(self):
        q = self.search_var.get().strip()
        for r in self.tree.get_children():
            self.tree.delete(r)
        if not q:
            self.load_tasks()
            return
        rows = self.db.search_tasks(q)
        for row in rows:
            tid, title, desc, due, priority, status = row
            self.tree.insert("", tk.END, values=(tid, title, due or "", priority or "", status or ""))
        self.info_label.config(text=f"Resultados encontrados: {len(rows)}")

    def get_selected_task_id(self):
        sel = self.tree.selection()
        if not sel:
            messagebox.showwarning("Atención","Seleccione primero una tarea.")
            return None
        return self.tree.item(sel[0])["values"][0]

    def open_add_form(self):
        TaskForm(self, self.db, on_save_callback=self.load_tasks)

    def open_edit_form(self):
        task_id = self.get_selected_task_id()
        if task_id:
            TaskForm(self, self.db, on_save_callback=self.load_tasks, task_id=task_id)

    def delete_selected(self):
        task_id = self.get_selected_task_id()
        if not task_id:
            return
        if messagebox.askyesno("Confirmar","¿Eliminar la tarea seleccionada?"):
            self.db.delete_task(task_id)
            self.load_tasks()
            messagebox.showinfo("Eliminado","Tarea borrada correctamente.")

    def mark_completed(self):
        task_id = self.get_selected_task_id()
        if not task_id:
            return
        record = self.db.get_task(task_id)
        if not record:
            messagebox.showerror("Error","No se encontró la tarea.")
            return
        self.db.update_task(task_id, record[1], record[2], record[3], record[4], "Completada")
        self.load_tasks()
        messagebox.showinfo("Actualizado","Tarea marcada como completada.")

    def open_about(self):
        AboutWindow(self, self.db)

    def on_close(self):
        self.db.close()
        self.destroy()

# ---------- PANTALLA DE BIENVENIDA ----------
def mostrar_bienvenida():
    def ingresar():
        splash.destroy()
        abrir_gestor()

    splash = tk.Tk()
    splash.title("Bienvenido")
    splash.geometry("500x300")
    splash.configure(bg="#8A2BE2")
    splash.resizable(False, False)

    splash.update_idletasks()
    width = splash.winfo_width()
    height = splash.winfo_height()
    x = (splash.winfo_screenwidth() // 2) - (width // 2)
    y = (splash.winfo_screenheight() // 2) - (height // 2)
    splash.geometry(f"{width}x{height}+{x}+{y}")

    # Título del programa
    titulo = tk.Label(
        splash,
        text="Task Manager - Gestor de Tareas Personales",
        bg="#8A2BE2",
        fg="white",
        font=("Helvetica", 16, "bold"),
        justify="center"
    )
    titulo.pack(pady=(20,10))

    # Mensaje de bienvenida
    mensaje = tk.Label(
        splash,
        text="¡Hola! Bienvenido al Gestor de Tareas\n\nPequeños avances diarios generan grandes resultados.",
        bg="#8A2BE2",
        fg="white",
        font=("Helvetica", 14),
        justify="center",
        wraplength=400
    )
    mensaje.pack(expand=True)

    # Botón ingresar
    btn_ingresar = ttk.Button(splash, text="Ingresar", command=ingresar)
    btn_ingresar.pack(pady=20)

    splash.mainloop()

# ---------- Función para abrir el Gestor ----------
def abrir_gestor():
    app = TaskManagerApp()
    app.protocol("WM_DELETE_WINDOW", app.on_close)
    app.mainloop()

# ---------- EJECUTAR APLICACIÓN ----------
if __name__ == "__main__":
    mostrar_bienvenida()
